In [1]:
# from english_words import english_words_set\
import multiprocessing as mp
import warnings
import pandas as pd
import numpy as np

In [2]:
warnings.filterwarnings('ignore')

In [3]:
f = open('guesses.txt','r')
guesses = []

for line in f:
    guesses.append(line.strip())

print(str(len(guesses)))

12972


In [4]:
f = open('solutions.txt','r')
solutions = []

for line in f:
    solutions.append(line.strip())

print(str(len(solutions)))

2315


In [5]:
words5 = solutions

In [6]:
# this should be an empty list, checking to be sure we do not have non alpha chars in the words
print([i for i in words5 if not i.isalpha()])

[]


In [7]:
len(words5)

2315

In [8]:
# word = 'wifie'
word_doubles = []
for word in words5:
    word_counts = 0
    for letter in word:
        # print(word)
        # print(letter)
        # print(str(word.count(letter)))
        word_counts += word.count(letter)
    if word_counts > 5:
        word_doubles.append(word)

In [9]:
len(word_doubles)

749

In [10]:
df_words = pd.DataFrame(words5)
df_words.columns=['words']

df_doubles = pd.DataFrame(word_doubles)
df_doubles.columns=['word']

In [11]:
df_doubles['double']=True

In [12]:
df_doubles.head()

,word,double
0,sissy,True
1,humph,True
2,awake,True
3,evade,True
4,naval,True


In [13]:
df_words['first'] = df_words['words'].str.slice(0,1)
df_words['second'] = df_words['words'].str.slice(1,2)
df_words['third'] = df_words['words'].str.slice(2,3)
df_words['fourth'] = df_words['words'].str.slice(3,4)
df_words['fifth'] = df_words['words'].str.slice(4,5)

In [14]:
df_words.head()

,words,first,second,third,fourth,fifth
0,cigar,c,i,g,a,r
1,rebut,r,e,b,u,t
2,sissy,s,i,s,s,y
3,humph,h,u,m,p,h
4,awake,a,w,a,k,e


In [15]:
df_words['first'][0]==df_words['second'][0]

False

In [16]:
aplhas = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [17]:
dfalpha = pd.DataFrame(aplhas)
dfalpha.columns = ['letters']
dfalpha.set_index('letters', inplace=True)

In [18]:
# build the frequerncy table from each smaller dataframe
df_counts = df_words['first'].value_counts()
df_counts_second = df_words['second'].value_counts()
df_counts_third = df_words['third'].value_counts()
df_counts_fourth = df_words['fourth'].value_counts()
df_counts_fifth = df_words['fifth'].value_counts()

dfranks = pd.concat([dfalpha, df_counts], axis=1)
dfranks = pd.concat([dfranks, df_counts_second], axis=1)
dfranks = pd.concat([dfranks, df_counts_third], axis=1)
dfranks = pd.concat([dfranks, df_counts_fourth], axis=1)
dfranks = pd.concat([dfranks, df_counts_fifth], axis=1)

In [19]:
df_counts.head()

s    366
c    198
b    173
t    149
p    142
Name: first, dtype: int64

In [20]:
dfranks.head()

,first,second,third,fourth,fifth
a,141.0,304,307,163.0,64.0
b,173.0,16,57,24.0,11.0
c,198.0,40,56,152.0,31.0
d,111.0,20,75,69.0,118.0
e,72.0,242,177,318.0,424.0


In [21]:
dfranks['first']['a']

141.0

In [22]:
def freq_ranking(word):
    val = dfranks['first'][word[0]] + dfranks['second'][word[1]] +dfranks['third'][word[2]] + dfranks['fourth'][word[3]] +dfranks['fifth'][word[4]]
    return val

In [23]:
df_word_ranks = pd.DataFrame({'word' : [],'value':[]})
word_freq = {'word':'value'}
for word in words5:
    df_word_ranks.loc[len(df_word_ranks.index), 'word'] = word
    df_word_ranks.loc[len(df_word_ranks.index)-1, 'value'] = freq_ranking(word)

In [24]:
df_word_ranks.head()

,word,value
0,cigar,842.0
1,rebut,739.0
2,sissy,1183.0
3,humph,505.0
4,awake,971.0


In [25]:
df_word_ranks = df_word_ranks.merge(df_doubles,how='left')
df_ranks_no_doubles = df_word_ranks.loc[df_word_ranks['double'] != True]

In [26]:
df_word_ranks.to_csv('word_ranks')

In [27]:
df_word_ranks.head()

,word,value,double
0,cigar,842.0,NaN
1,rebut,739.0,NaN
2,sissy,1183.0,True
3,humph,505.0,True
4,awake,971.0,True


In [28]:
print('with doubles')
print (df_word_ranks['word'][df_word_ranks['value'].idxmax()])
print (df_word_ranks['value'][df_word_ranks['value'].idxmax()])
print('no double letters')
print (df_ranks_no_doubles['word'][df_ranks_no_doubles['value'].idxmax()])
print (df_ranks_no_doubles['value'][df_ranks_no_doubles['value'].idxmax()])

with doubles
slate
1437.0
no double letters
slate
1437.0


In [29]:
# here we determine the next best word based on results of the best word per wordle feedback
# how to determine which items were elimiated
#  ~ before a contains statement makes it a "not" I was lucky to find this
display(df_word_ranks.loc[(~df_word_ranks['word'].str.contains('t'))])

,word,value,double
0,cigar,842.0,NaN
2,sissy,1183.0,True
3,humph,505.0,True
4,awake,971.0,True
5,blush,849.0,NaN
...,...,...,...
2305,rival,675.0,NaN
2310,judge,781.0,NaN
2311,rower,940.0,True
2313,rural,773.0,True


In [30]:
grey = ['s','l','a','t','e']

In [31]:
df_sub = df_word_ranks
for l in grey:
    # print(l)
    # print(str(len(df_sub.index)))
    df_sub = (df_sub.loc[(~df_sub['word'].str.contains(l))])
df_sub

,word,value,double
3,humph,505.0,True
48,mimic,559.0,True
49,pound,886.0,NaN
54,booby,1084.0,True
59,ivory,809.0,NaN
...,...,...,...
2255,going,883.0,True
2258,hound,813.0,NaN
2267,undid,471.0,True
2283,knock,616.0,True


In [32]:
print (df_sub['word'][df_sub['value'].idxmax()])
print (df_sub['value'][df_sub['value'].idxmax()])

crony
1255.0


In [33]:
def remove_notcontains(grey,df):
    df_sub = df
    for l in grey:
        df_sub = (df_sub.loc[(~df_sub['word'].str.contains(l))])
    return df_sub

In [34]:
def remove_by_contains(letters,df):
    df_sub = df
    for l in letters:
        df_sub = (df_sub.loc[(df_sub['word'].str.contains(l))])
    return df_sub

In [35]:
def remove_by_location(loc,letter,df):
    df_sub = df
    for l in letter:
        df_sub = (df_sub.loc[(df_sub['word'].str[loc]==l)])
    return df_sub

In [36]:
def remove_by_letter_count(letter,count,df):
    df_sub = df
    for l in letter:
        df_sub = (df_sub.loc[(df_sub['word'].str.count(letter)==count)])
    return df_sub

In [37]:
confirmed = 'cr'

In [38]:
test = "dogg"
print(test.count('g'))

2


In [39]:
word = 'score'
guess = 'scres'
score = 0

for i in range(0,4):
    # find character match in locations
    if guess[i]==word[i]:
        print(f'exact char match on letter {letter} in positions {i}')
        score += 2

exact char match on letter e in positions 0
exact char match on letter e in positions 1


In [40]:
solution = ['write','fight','soyet','cleat']

df_geuss_value = pd.DataFrame({'guess' : [],'score':[], 'answer':[]})

for idx, word in enumerate(solution):
    print(str(idx))
    print(str(word))

    df_geuss_value.loc[idx, 'guess'] = guess
    df_geuss_value.loc[idx, 'score'] = score
    df_geuss_value.loc[idx, 'answer'] = word

df_geuss_value

0
write
1
fight
2
soyet
3
cleat


,guess,score,answer
0,scres,4.0,write
1,scres,4.0,fight
2,scres,4.0,soyet
3,scres,4.0,cleat


In [41]:
df_geuss_value.head()

,guess,score,answer
0,scres,4.0,write
1,scres,4.0,fight
2,scres,4.0,soyet
3,scres,4.0,cleat


without parallels

In [42]:
# now try this for all words in the guesses table vs all the potential solutions
# calcuating potential eliminations for each starting word and each potential answer
# solution = ['write','fight','soyet','cleat']

# guess = ''

# # for guess in solutions:
# for guess in few_guesses:

#     df_geuss_value = pd.DataFrame({'guess':[],'score_match':[], 'answer':[], 'score_exact_match':[]})

#     for idx, word in enumerate(solutions):
#         score = 0
#         score_exact = 0
#         # print(word)
#         for letter in guess:
#     # match of letter
#     # give one point
#             # print(f' for the guess {guess}, and the answer {word} the score for letter: {letter} is {word.count(letter)}')
#             score += int(word.count(letter))
#         if word.count(letter) >=1:
#             for i in range(0,4):
#     # find character match in locations
#                 char = guess[i]
#                 if guess[i]==word[i]:
#                     # print(f'exact char match on letter {char} in positions {i}')
#                     score_exact += 1
        
#         df_geuss_value.loc[idx, 'guess'] = guess
#         df_geuss_value.loc[idx, 'score_match'] = score
#         df_geuss_value.loc[idx, 'answer'] = word
#         df_geuss_value.loc[idx, 'score_exact_match'] = score_exact


In [43]:
df_geuss_value = pd.DataFrame({'guess':[],'score_match':[], 'answer':[], 'score_exact_match':[]})

In [44]:
print(df_geuss_value.head())
index = df_geuss_value.index
number_of_rows = len(index)
print(number_of_rows)
df_geuss_value.to_csv('guess_value_iterations_guesses')

Empty DataFrame
Columns: [guess, score_match, answer, score_exact_match]
Index: []
0


todo - with parallel threads

In [45]:
# turn my existing loop into a function that can be parallelized
def word_val(guess, solution_list):
    df_geuss_value = pd.DataFrame({'guess':[],'score_match':[], 'answer':[], 'score_exact_match':[]})


# make a smaller dataframe for each guess and have that be appended to the larger dataframe for each individual guess


    for idx, word in enumerate(solution_list):
        results = []
        score = 0
        score_exact = 0
        # print(word)
        for letter in guess:
    # match of letter
    # give one point
            # print(f' for the guess {guess}, and the answer {word} the score for letter: {letter} is {word.count(letter)}')
            score += int(word.count(letter))
        if word.count(letter) >=1:
            for i in range(0,4):
    # find character match in locations
                char = guess[i]
                if guess[i]==word[i]:
                    # print(f'exact char match on letter {char} in positions {i}')
                    score_exact += 1
        
        df_geuss_value.loc[idx, 'guess'] = guess
        df_geuss_value.loc[idx, 'score_match'] = score
        df_geuss_value.loc[idx, 'answer'] = word
        df_geuss_value.loc[idx, 'score_exact_match'] = score_exact

    # results.append(guess)
    # results.append(score)
    # results.append(word)
    # results.append(score_exact)
    # results.append(idx)

    return df_geuss_value



In [46]:
def row_results(results_df):
    # global df_geuss_value
    # df_geuss_value.loc[idx, 'guess'] = results[0]
    # df_geuss_value.loc[idx, 'score_match'] = results[1]
    # df_geuss_value.loc[idx, 'answer'] = results[2]
    # df_geuss_value.loc[idx, 'score_exact_match'] = results[3]
    global df_geuss_value
    df_geuss_value = df_geuss_value.append(results_df)
    # df_geuss_value = pd.concat(results_df)


In [47]:
few_guesses = guesses[0:50]
# few_guesses

In [48]:
# warnings.filterwarnings(action='once')

# for idx, guess in enumerate(few_guesses):
#     # print(guess)
#     # print(word_val(guess,solutions))
#     row_results(word_val(guess,solutions))

In [52]:
print(df_geuss_value.head())
index = df_geuss_value.index
number_of_rows = len(index)
print(number_of_rows)
df_geuss_value.to_csv('guess_value_iterations_guesses')

Empty DataFrame
Columns: [guess, score_match, answer, score_exact_match]
Index: []
0


In [50]:
df_geuss_value = pd.DataFrame({'guess':[],'score_match':[], 'answer':[], 'score_exact_match':[]})

In [51]:
print(f'Number of CPUs availible: {mp.cpu_count()}')
# pool = mp.Pool(mp.cpu_count())
pool = mp.Pool(8)

Number of CPUs availible: 16


In [53]:
for guess in guesses:
    pool.apply_async(word_val, args=(guess,solutions),callback=row_results)

pool.close()
pool.join()

In [54]:
print(df_geuss_value.head())
index = df_geuss_value.index
number_of_rows = len(index)
print(number_of_rows)
df_geuss_value.to_csv('guess_value_iterations_guesses')

   guess  score_match answer  score_exact_match
0  cigar          5.0  cigar                4.0
1  cigar          1.0  rebut                0.0
2  cigar          1.0  sissy                0.0
3  cigar          0.0  humph                0.0
4  cigar          2.0  awake                0.0
30030180


In [ ]:
# now try this for all words in the guesses table vs all the potential solutions
# calcuating potential eliminations for each starting word and each potential answer
# solution = ['write','fight','soyet','cleat']

guess = ''

for guess in solutions[0:50]:
# for guess in guesses:

    df_geuss_value = pd.DataFrame({'guess':[],'score_match':[], 'answer':[], 'score_exact_match':[]})

    for idx, word in enumerate(solutions):
        score = 0
        score_exact = 0
        # print(word)
        for letter in guess:
    # match of letter
    # give one point
            # print(f' for the guess {guess}, and the answer {word} the score for letter: {letter} is {word.count(letter)}')
            score += int(word.count(letter))
        if word.count(letter) >=1:
            for i in range(0,4):
    # find character match in locations
                char = guess[i]
                if guess[i]==word[i]:
                    # print(f'exact char match on letter {char} in positions {i}')
                    score_exact += 1
        
        df_geuss_value.loc[idx, 'guess'] = guess
        df_geuss_value.loc[idx, 'score_match'] = score
        df_geuss_value.loc[idx, 'answer'] = word
        df_geuss_value.loc[idx, 'score_exact_match'] = score_exact

<-- below this is the game aid loop -->

here is the game aid loop

In [ ]:
# make the testing loop
# starting DF df_word_ranks

round = 0
df_options = df_word_ranks

print('strongest suggestions')
print(df_options.nlargest(3, 'value'))
go = 'go'

while go != 'no':
    round += 1 
    if round == 1:

        print('initial guess')
        word = input('enter your guess:')
        pass

    if round > 1:

        print(str(round))

        not_char = input('enter eliminated letters:[l,e,t,t,e,r]')
        is_char = input('enter confirmed letters:[y,e,s]')
        loc_letters = input('enter confirmed locations: [letter,location]')
        dup_letters = input('enter known letter counts: [letter,count]')

        # eliminate options, and show the reduced best 5 choices
        if len(not_char)>0:
            df_options = remove_notcontains(not_char,df_options)

        if len(is_char)>0:
            df_options = remove_by_contains(is_char,df_options)

        if len(loc_letters)>0:
            # remove by letters in correct location
            for i in range(0,int(len(loc_letters)),2):
                df_options = remove_by_location(loc=int(loc_letters[i+1]),letter=str(loc_letters[i]),df=df_options)
        
        if len(dup_letters)>0:
            # remove by letters in correct location
            for i in range(0,int(len(loc_letters)),2):
                df_options = remove_by_letter_count(loc=int(dup_letters[i+1]),letter=str(dup_letters[i]),df=df_options)
        
        try:
            next_best = (df_options['word'][df_options['value'].idxmax()])
            print('from the results of the last guess')
            print('next best option is ' + str(next_best))
            print('your top remaiing options are: ')
            print(df_options.nlargest(3, 'value'))
        except:
            print('e message')
            print(df_options.head())
            pass
    
        go = input('go again?:')
    # go = input('go again?:')
    print(str(round))


In [ ]:
#  this can be improved by confiming the number of letters in a word, 'wince' has one E and this was confirmed
# use this to elimate more from the pool when the information is availible